### Import Dependencies

In [1]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PayloadSchemaType, PointStruct, SparseVectorParams, Document, Prefetch, FusionQuery
from qdrant_client import models

import pandas as pd
import openai
import cohere
import os

/Users/hanlinyang/Projects/AI/ai_bootcamp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Retrieval

In [2]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [3]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=[text],
        model=model,
    )
    return response.data[0].embedding

In [4]:
def retrieve_data(query, qdrant_client, k=5):
    query_embedding = get_embedding(query)
    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-01-hybrid-search-v2",
        prefetch=[
            Prefetch(
                query=query_embedding,
                using="text-embedding-3-small",
                limit=20
            ),
            Prefetch(
                query=Document(text=query, model="Qdrant/bm25"),
                using="bm25",
                limit=20
            )
        ],
        query=FusionQuery(fusion="rrf"),
        limit=k
    )

    retrieved_context_ids = []
    retrieved_context= []
    similiarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload['parent_asin'])
        retrieved_context.append(result.payload['description'])
        similiarity_scores.append(result.score)
        retrieved_context_ratings.append(result.payload['average_rating'])

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "similiarity_scores": similiarity_scores,
        "retrieved_context_ratings": retrieved_context_ratings
    }

In [5]:
query = "I want to buy a tablet"

In [6]:
results = retrieve_data(query, qdrant_client, k=20)

In [7]:
results

{'retrieved_context_ids': ['B0BBFVQLVQ',
  'B0C697WS8L',
  'B07G3WVR5C',
  'B0B2RK8PHB',
  'B0CF1B9QTM',
  'B09TPD8R5F',
  'B0B8PBZH45',
  'B09SW5L2PV',
  'B0BCDYMF5L',
  'B0CC94G5J9',
  'B0BTHP8WP5',
  'B09TSQXTYZ',
  'B0BRM8DC9H',
  'B0B4SN9K1T',
  'B09TH8S2WT',
  'B0BK935JV7',
  'B09PTSR7TK',
  'B0BCF1BSXK',
  'B0BJSGG7M2',
  'B09QS7Z919'],
 'retrieved_context': ["AZXRHWYGS Projector Screen with Stand,100 Inch Foldable Portable Projector Screen 16：9 HD 4K Double Sided Projection Movie Screen. Projector Screen with Stand -- The projection screen has made of high-quality polyester & spandex material, it is flexible and smooth, without traces and no crease after folded,can provide brighter colors and clearer images, Also, this fabric will not be deformed after washing, easy to clean. The stand has made of durable steel to keep stable.Front/Rear Projection -- The 100inch projection screen supports front and rear projection, you can see clear pictures on any side. Put the projector on th

### Reranking

In [8]:
cohere_client = cohere.ClientV2(api_key=os.environ["CO_API_KEY"])

In [9]:
to_rerank = results['retrieved_context']

In [16]:
to_rerank

["AZXRHWYGS Projector Screen with Stand,100 Inch Foldable Portable Projector Screen 16：9 HD 4K Double Sided Projection Movie Screen. Projector Screen with Stand -- The projection screen has made of high-quality polyester & spandex material, it is flexible and smooth, without traces and no crease after folded,can provide brighter colors and clearer images, Also, this fabric will not be deformed after washing, easy to clean. The stand has made of durable steel to keep stable.Front/Rear Projection -- The 100inch projection screen supports front and rear projection, you can see clear pictures on any side. Put the projector on the opposite side, do not need to worry about people walking around will affect the movie. 160° Wide Viewing Angle, 100-inch Diagonal，Gain 1.1 and 16:9 aspect ratio，support for multiple viewers and not crowded.Lightweight And Easy To Carry -- The projector screen with stand 100 inch has weight of 6 lbs and come with a carry bag,easy to carry in anywhere you want and p

#### Test the best model so far

In [18]:
response = cohere_client.rerank(
    model="rerank-v4.0-pro",
    query=query,
    documents=to_rerank,
    top_n=20
)

In [12]:
response

V2RerankResponse(id='1fee92c0-11d2-4157-b604-d9694250d763', results=[V2RerankResponseResultsItem(index=1, relevance_score=0.9042201), V2RerankResponseResultsItem(index=19, relevance_score=0.89297384), V2RerankResponseResultsItem(index=6, relevance_score=0.89222497), V2RerankResponseResultsItem(index=3, relevance_score=0.89099807), V2RerankResponseResultsItem(index=8, relevance_score=0.88347745), V2RerankResponseResultsItem(index=4, relevance_score=0.88246834), V2RerankResponseResultsItem(index=11, relevance_score=0.8730417), V2RerankResponseResultsItem(index=16, relevance_score=0.8704211), V2RerankResponseResultsItem(index=15, relevance_score=0.8585289), V2RerankResponseResultsItem(index=13, relevance_score=0.85322917), V2RerankResponseResultsItem(index=10, relevance_score=0.747321), V2RerankResponseResultsItem(index=5, relevance_score=0.71827835), V2RerankResponseResultsItem(index=2, relevance_score=0.69064254), V2RerankResponseResultsItem(index=14, relevance_score=0.67542213), V2Rera

In [13]:
reranked_results = [to_rerank[result.index] for result in response.results]

In [14]:
reranked_results

["2023 Newest Android 11.0 Tablet, 2 in 1 Tablet 10 Inch, 5G Dual WIFI Tablet with Keyboard, 64GB ROM + 6GB RAM, 256GB Expand, Quad-Core Processor, 2 Card Slot, 13MP Camera, WIFI/ Bluetooth/OTG - Silver. 【Powerful System】AOYODKG 2023 android tablet uses Google's certified Android 11.0 operating system and can access Google Play, 10 inch tablet allowing you to download the applications you need. Equipped with quad-core 64-bit Cortex- A55 1.8-2.3Ghz, enables quicker app launch, smoother videos and better overall performance.【Ultra-clean Record】The android tablet with keyboard bundle high clear tablet PC come with 13mp dual camera can take beautiful photos and video conversations, stylish appearance, High resolution of 10 inch 1280 * 800 IPS display. the cheap android tablet 10 inch best choice for movie and playing game.【Larger and Larger Capacity】Equipped with 6GB of running memory and runs faster. 64GB high-speed storage, combined with a micro SD card slot, can expand the memory to 256

#### Test a ligher model

In [19]:
response_v2 = cohere_client.rerank(
    model="rerank-v4.0-fast",
    query=query,
    documents=to_rerank,
    top_n=20
)

In [20]:
reranked_results_v2 = [to_rerank[result.index] for result in response_v2.results]

In [21]:
reranked_results_v2

["2023 Newest Android 11.0 Tablet, 2 in 1 Tablet 10 Inch, 5G Dual WIFI Tablet with Keyboard, 64GB ROM + 6GB RAM, 256GB Expand, Quad-Core Processor, 2 Card Slot, 13MP Camera, WIFI/ Bluetooth/OTG - Silver. 【Powerful System】AOYODKG 2023 android tablet uses Google's certified Android 11.0 operating system and can access Google Play, 10 inch tablet allowing you to download the applications you need. Equipped with quad-core 64-bit Cortex- A55 1.8-2.3Ghz, enables quicker app launch, smoother videos and better overall performance.【Ultra-clean Record】The android tablet with keyboard bundle high clear tablet PC come with 13mp dual camera can take beautiful photos and video conversations, stylish appearance, High resolution of 10 inch 1280 * 800 IPS display. the cheap android tablet 10 inch best choice for movie and playing game.【Larger and Larger Capacity】Equipped with 6GB of running memory and runs faster. 64GB high-speed storage, combined with a micro SD card slot, can expand the memory to 256